**Prediction Time Series Data using Linear Regression and RandomForestRegressor**

Approach:

 - Clean the data
 - Remove outliers
 - Build a model
 - Apply the model to test data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from dateutil.parser import parse 
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['text.color'] = 'k'

In [ ]:
item_categories=pd.read_csv(r'../input/competitive-data-science-predict-future-sales/item_categories.csv')
items=pd.read_csv(r'../input/competitive-data-science-predict-future-sales/items.csv')
raw_sales=pd.read_csv(r'../input/competitive-data-science-predict-future-sales/sales_train.csv')
shops=pd.read_csv(r'../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
sample=pd.read_csv(r'../input/competitive-data-science-predict-future-sales/sample_submission.csv')
sample

In [ ]:
sales1 = pd.merge(raw_sales, items, on ='item_id',how ='left')
sales2 = pd.merge(sales1, item_categories, on ='item_category_id',how ='left')
sales3 = pd.merge(sales2, shops, on ='shop_id',how ='left')
sales3

In [ ]:
sales3.isna().sum()

**Check for Correlations**

In [ ]:
y=sales3[['date', 'item_cnt_day']]
y.head()

In [ ]:
y = y.groupby('date')['item_cnt_day'].sum().reset_index()
y.head()

In [ ]:
y.shape[0]

In [ ]:
for i in range(0,y.shape[0]):
  y['date'][i] = pd.concat(y['date'][i].str[-4:], y['date'][i].str[3:4], y['date'][i].str[0:2])

In [ ]:
pd.concat([y['date'].str[-4:], y['date'].str[3:5], y['date'].str[0:2]])

In [ ]:
y['date'].str[3:5]

In [ ]:
y['date'].str[0:2]

In [ ]:
y['date'] = pd.concat(y['date'].str[-4:], y['date'].str[3:4], y['date'].str[0:2])

In [ ]:
y.set_index('date')
y.index

In [ ]:
y

In [ ]:
x = y['item_cnt_day'].resample('MS').mean()

In [ ]:
y.plot(figsize=(15, 6))
plt.show()

In [ ]:
sns.heatmap(sales3.corr())

**Sumif the count per days according to months, items & shops**

In [ ]:
cnt_by_month=sales3.groupby('date_block_num').sum()
cnt_by_month=cnt_by_month.drop(['shop_id', 'item_id', 'item_price', 'item_category_id'], axis=1)
cnt_by_month.reset_index(inplace=True)

cnt_by_item=sales3.groupby('item_id').sum()
cnt_by_item=cnt_by_item.drop(['shop_id', 'date_block_num', 'item_price', 'item_category_id'], axis=1)
cnt_by_item.reset_index(inplace=True)

cnt_by_shop=sales3.groupby('shop_id').sum()
cnt_by_shop=cnt_by_shop.drop(['item_id', 'date_block_num', 'item_price', 'item_category_id'], axis=1)
cnt_by_shop.reset_index(inplace=True)

**Plot the data to see the outliers**

In [ ]:
fig1 = sns.relplot(x='date_block_num', y='item_cnt_day', data=cnt_by_month)
fig1a = fig1.fig 
fig1a.suptitle("Total solds Time Series", fontsize=12)


fig2 = sns.relplot(x='item_id', y='item_cnt_day', data=cnt_by_item)
fig2a = fig2.fig 
fig2a.suptitle("Items sold", fontsize=12)


fig3 = sns.relplot(x='shop_id', y='item_cnt_day', data=cnt_by_shop)
fig3a = fig3.fig 
fig3a.suptitle("Solds per Shop", fontsize=12)

In [ ]:
print('Best Selling Months:')
date_outlier=[i for i in cnt_by_month['item_cnt_day'] if i > 160000]
date_outlier1=pd.DataFrame({"item_cnt_day": date_outlier})
date_outlier2 = pd.merge(date_outlier1, cnt_by_month, on ='item_cnt_day', how ='inner')
date_outlier2

In [ ]:
sales3.loc[sales3['date_block_num'] == 11, 'date']

In [ ]:
sales3.loc[sales3['date_block_num'] == 23, 'date']

In [ ]:
#Peak sales were before Winter Holidays, seasonal outliers

In [ ]:
print('Best Selling Items:')
item_outlier=[i for i in cnt_by_item['item_cnt_day'] if i > 150000]
item_outlier1=pd.DataFrame({"item_cnt_day": item_outlier})
item_outlier2 = pd.merge(item_outlier1, cnt_by_item, on ='item_cnt_day', how ='inner')
item_outlier2

In [ ]:
print('Best Selling Shops:')
shop_outlier=[i for i in cnt_by_shop['item_cnt_day'] if i > 180000]
shop_outlier1=pd.DataFrame({"item_cnt_day": shop_outlier})
shop_outlier2 = pd.merge(shop_outlier1, cnt_by_shop, on ='item_cnt_day', how ='inner')
shop_outlier2

In [ ]:
sales4=sales3.copy()
sales4 = sales4[(sales4.date_block_num != 11) & (sales4.date_block_num != 23)]
sales4 = sales4[sales4.item_id != 20949]
sales4 = sales4[(sales4.shop_id != 25) & (sales4.shop_id != 28) & (sales4.shop_id != 31 & (sales4.shop_id != 54))]

In [ ]:
sales4

In [ ]:
sales5=sales4.drop(['date', 'item_name', 'item_category_name', 'shop_name'], axis=1)
sales5

In [ ]:
sales6=sales5.copy()
sales6=sales6.drop(['item_price', 'item_category_id'], axis=1)

In [ ]:
sales6 = sales6.groupby(['date_block_num','shop_id','item_id']).sum()
sales6.reset_index(inplace=True)

In [ ]:
sales6

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
sales6[['item_cnt_day']] = scale.fit_transform(sales6[['item_cnt_day']])

In [ ]:
sales7 = sales6.pivot_table(index=['shop_id','item_id'], columns='date_block_num', values='item_cnt_day', fill_value=0)
sales7.reset_index(inplace=True)

In [ ]:
sales7

In [ ]:
sales8 = sales6.groupby(['shop_id','item_id']).sum()
sales8.reset_index(inplace=True)

In [ ]:
sales8

In [ ]:
sales7['mean'] = sales7.iloc[:, 2:].mean(axis=1)

In [ ]:
sales7['mean']

In [ ]:
x=sales7.drop('mean', axis=1)
y=sales7['mean']

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

**Build the Linear Regression Model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_train, model.predict(X_train))

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RFR = RandomForestRegressor(n_estimators = 10)
RFR.fit(X_train,y_train)

In [ ]:
RFR.score(X_train,y_train)

In [ ]:
y_pred = RFR.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
test=pd.read_csv(r'../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
test

In [ ]:
test1=test.merge(x, how="left", on = ['shop_id', 'item_id'])

In [ ]:
test1

In [ ]:
test2=test1.copy()
test2=test2.fillna(0)
test2=test2.drop('ID', axis=1)
test2

In [ ]:
y_pred1=RFR.predict(test2)

In [ ]:
y_new_inverse = scale.inverse_transform(y_pred1)

In [ ]:
y_pred2=pd.DataFrame(y_new_inverse, columns=['item_cnt_month'])
y_pred2

In [ ]:
submission=pd.concat([test['ID'], y_pred2], axis=1)

**Export the outcome**

In [ ]:
submission.to_csv(r'./submission.csv', index = False)